In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

2023-04-04 11:59:00.154544: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def load_dataset(dsname):
    metadata = pd.read_csv(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/metadata_final.csv")
    x = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_data0.p", "rb" ))
    y = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_labels.p", "rb" ))
    return metadata, x, y

In [3]:
from sklearn.preprocessing import StandardScaler

datasets = ["emodb", "emovo", "ravdess"]
scalers = {d: StandardScaler() for d in datasets}

data = {}
for d in datasets:
    md, x, y = load_dataset(d)
    data[d] = {}
    # standardize data
    n = x.shape
    data[d]["x"] = scalers[d].fit_transform(x.reshape(n[0],-1)).reshape(n)
    data[d]["y"] = y
    data[d]["metadata"] = md

In [4]:
from sklearn.model_selection import train_test_split

train = {}
test = {}
test_md = {}
for d in datasets:
    train[d] = {}
    test[d] = {}
    train_df, test_df = train_test_split(data[d]['metadata'], 
                                       test_size = 0.2, 
                                       random_state = 1938,
                                       stratify = data[d]['metadata']['label'])
    test_md[d] = test_df.reset_index()
    train_index = train_df.index
    test_index = test_df.index
    print(f'For {d}:\ttrain size {len(train_index)}, test size {len(test_index)}')
    train[d]['x'] = data[d]['x'][train_index]
    train[d]['y'] = data[d]['y'][train_index]
    test[d]['x']= data[d]['x'][test_index]
    test[d]['y']= data[d]['y'][test_index]

For emodb:	train size 3344, test size 837
For emovo:	train size 3944, test size 987
For ravdess:	train size 7161, test size 1791


In [5]:
def get_m_f_data(md):
    m = md[md.gender=='m'].index
    f = md[md.gender=='f'].index
    return m, f

test_mf = {}
for d in datasets:
    data_d = get_m_f_data(test_md[d])
    print(f"For {d}\n\tmale:{data_d[0].shape[0]}\n\tfemale:{data_d[1].shape[0]}\n")

For emodb
	male:315
	female:522

For emovo
	male:488
	female:499

For ravdess
	male:895
	female:896



In [6]:
models = {d: tf.keras.models.load_model(f'/datasets/nicolas_facchinetti/models/{d}')
          for d in datasets}

2023-04-04 11:59:08.898676: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-04 11:59:08.899499: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-04 11:59:09.016282: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 607.97GiB/s
2023-04-04 11:59:09.016314: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-04 11:59:09.017690: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-04 11:59:09.017770: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2023-0

In [7]:
import keras_tuner as kt
param = pickle.load(open('/datasets/nicolas_facchinetti/param.p', "rb" ))

# ART Model

Find max and min value for each dataset to use as clip values for attacks (strictly needed for black box)

In [8]:
minv = np.array([data[d]['x'].min() for d in models])
minv = {d: v for v, d in zip(np.floor(minv), models)}

maxv = np.array([data[d]['x'].max() for d in models])
maxv = {d: v for v, d in zip(np.ceil(maxv), models)}
minv, maxv

({'emodb': -5.0, 'emovo': -3.0, 'ravdess': -2.0},
 {'emodb': 7.0, 'emovo': 7.0, 'ravdess': 8.0})

Make ART model

In [9]:
from art.estimators.classification import TensorFlowV2Classifier

def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

classifiers = {d: TensorFlowV2Classifier(models[d],
                                         nb_classes=5,
                                         loss_object=loss_object,
                                         train_step=train_step,
                                         input_shape=(261,128,1),
                                         clip_values=(minv[d], maxv[d]),
                                         channels_first=False)
               for d in models}

/home/nicolas/.conda/envs/thesis/lib/python3.9/site-packages/art/estimators/certification/__init__.py:14: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


# Adversarial training

In [10]:
from art.defences.trainer import AdversarialTrainer
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import DeepFool
from art.attacks.evasion import CarliniLInfMethod
from art.attacks.evasion import SaliencyMapMethod
from art.attacks.evasion import BoundaryAttack
from art.attacks.evasion import PixelAttack


adv_models = {}

def get_adv_model(d):
    conf = FastGradientMethod(estimator=classifiers[d],
                              eps=1.25,
                              batch_size=256)
    
    
    return AdversarialTrainer(classifiers[d], attacks=[conf], ratio=0.6)

In [11]:
adv_models['emodb'] = get_adv_model('emodb')
adv_models['emovo'] = get_adv_model('emovo')
adv_models['ravdess'] = get_adv_model('ravdess')

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_delta=1e-4, mode='min')
    
new_models = {}

def evaluate(d, m):
    mi, fi = get_m_f_data(test_md[d])
    loss, accuracy = m.evaluate(test[d]['x'], test[d]['y'], verbose=0)
    print(f'\tAll data\ttest loss: {loss:.6f} accuracy: {accuracy:.6f}')
    lossm, accuracym = m.evaluate(test[d]['x'][mi], test[d]['y'][mi], verbose=0)
    print(f'\tMale data\ttest loss: {lossm:.6f} accuracy: {accuracym:.6f}')
    lossf, accuracyf = m.evaluate(test[d]['x'][fi], test[d]['y'][fi], verbose=0)
    print(f'\tFemale data\ttest loss: {lossf:.6f} accuracy: {accuracyf:.6f}\n')

for d in adv_models:
    print(f"\n\nTraining on {d}")
    
    evaluate(d, adv_models[d].get_classifier().get_params()['model'])
    
    adv_models[d].fit(train[d]['x'], train[d]['y'],
                      batch_size=128,
                      nb_epochs=30,
                      validation_split=0.2,
                      callbacks=[earlyStopping, reduce_lr_loss])
    new_models[d] = adv_models[d].get_classifier().get_params()['model']
    
    evaluate(d, new_models[d])



Training on emodb


2023-04-04 11:59:21.222854: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-04 11:59:21.223561: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3301490000 Hz
2023-04-04 11:59:21.665103: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-04 11:59:21.860404: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


	All data	test loss: 0.316978 accuracy: 0.909200
	Male data	test loss: 0.353135 accuracy: 0.895238
	Female data	test loss: 0.295158 accuracy: 0.917625



Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/30 [00:00<?, ?it/s]

...

Adversarial training epochs:   0%|          | 0/30 [00:00<?, ?it/s]

	All data	test loss: 0.319508 accuracy: 0.902736
	Male data	test loss: 0.264935 accuracy: 0.913934
	Female data	test loss: 0.372879 accuracy: 0.891784



Training on ravdess
	All data	test loss: 0.313163 accuracy: 0.910664
	Male data	test loss: 0.307363 accuracy: 0.910615
	Female data	test loss: 0.318956 accuracy: 0.910714



Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/30 [00:00<?, ?it/s]

	All data	test loss: 0.250914 accuracy: 0.919598
	Male data	test loss: 0.232654 accuracy: 0.918436
	Female data	test loss: 0.269153 accuracy: 0.920759



# Evaluate impact

Model attack function

In [13]:
import time

def do_attack(method, X):
    start = time.time()
    x_adv = method.generate(X, verbose=True)
    elapsed = time.time() - start
    print(f'Attack get {elapsed} seconds to complete')
    return x_adv, elapsed

In [14]:
def evaluate_attack(model, test_samples, test_metadata, x_adv):
    mi, fi = get_m_f_data(test_metadata)
    x = test_samples['x']
    y = test_samples['y']
    
    def evaluate_on_samples(model, adv, x, y):
        loss, accuracy = model.evaluate(adv, y, verbose=0)
        perturbation = np.mean(np.abs((adv - x)))
        print(f'\t\tloss: {loss:.5f}, accuracy: {accuracy:.5f}, avg perturbation: {perturbation:.5f}\n')
        return loss, accuracy, perturbation
    
    print('\tWhole test set')
    l1, a1, p1 = evaluate_on_samples(model, x_adv, x, y)
    print('\tOnly male data')
    l2, a2, p2 = evaluate_on_samples(model, x_adv[mi], x[mi], y[mi])
    print('\tOnly female data')
    l3, a3, p3 = evaluate_on_samples(model, x_adv[fi], x[fi], y[fi])
    
    def make_dict(v):
        return {'All': v[0], 'Male': v[1], 'Female': v[2]}
    
    res_dict = {'Loss': make_dict([l1,l2,l3]),
                'Accuracy': make_dict([a1,a2,a3]),
                'Perturbation': make_dict([p1,p2,p3])}
    return res_dict

In [15]:
def attack(attacks_confs, name):
    # save results for dataframe output
    results[name] = {}
    times[name] = {}
    
    # return to store adv generated
    adv_generated = {}
    adv_generated[name] = {}
    for d in attacks_confs:
        results[name][d] = {}
        times[name][d] = {}
        adv_generated[name][d] = {}
        for p in attacks_confs[d]:
            print(f'Attacking {d} with {name}, parameter {p}')
            
            # compute adv examples and get time
            x_adv, time = do_attack(attacks_confs[d][p], test[d]['x'])
            # test adv examples on whole data, only M, and only F
            res = evaluate_attack(new_models[d], test[d], test_md[d], x_adv)
            
            results[name][d][p] = res
            times[name][d][p] = time
            adv_generated[name][d][p] = x_adv
            
        print("_____________________________")
    return adv_generated

Functions to display data in DataFrame

In [16]:
def df_acc(d_res):
    reform = {(dataset, param): values
              for dataset, inner1 in d_res.items()
              for param, inner2 in inner1.items()
              for metric, values in inner2.items()
              if metric == 'Accuracy'}
    mux = pd.MultiIndex.from_tuples(reform.keys())
    df = pd.DataFrame(list(reform.values()), index=mux)
    return df.unstack(level=1).stack(level=0)

In [17]:
def save_adv(data):
    for attack in data:
        pickle.dump(data[attack],
                    open(f'/datasets/nicolas_facchinetti/adv2/{attack}.p', "wb"))
    

In [18]:
# size of the batch on which adversarial samples are generated.
bs = 256
# store accuracy, loss and perturbation levels
results = {}
# store time to generate the samples
times = {}

Various attacks

In [19]:
epss = [0.25, 0.5, 0.75, 1, 1.25]
conf1 = {d: {e: FastGradientMethod(estimator=classifiers[d],
                                   eps=e,
                                   batch_size=bs)
             for e in epss}
         for d in datasets}

adv = attack(conf1, 'fgsm')

Attacking emodb with fgsm, parameter 0.25
Attack get 1.7493360042572021 seconds to complete
	Whole test set
		loss: 5.16057, accuracy: 0.15532, avg perturbation: 0.21631

	Only male data
		loss: 6.04225, accuracy: 0.09206, avg perturbation: 0.21616

	Only female data
		loss: 4.62852, accuracy: 0.19349, avg perturbation: 0.21640

Attacking emodb with fgsm, parameter 0.5
Attack get 1.1954381465911865 seconds to complete
	Whole test set
		loss: 2.42037, accuracy: 0.31780, avg perturbation: 0.43262

	Only male data
		loss: 2.69087, accuracy: 0.25079, avg perturbation: 0.43231

	Only female data
		loss: 2.25714, accuracy: 0.35824, avg perturbation: 0.43280

Attacking emodb with fgsm, parameter 0.75
Attack get 1.1911406517028809 seconds to complete
	Whole test set
		loss: 1.09087, accuracy: 0.63321, avg perturbation: 0.64893

	Only male data
		loss: 1.01887, accuracy: 0.65397, avg perturbation: 0.64847

	Only female data
		loss: 1.13431, accuracy: 0.62069, avg perturbation: 0.64920

Attackin

In [20]:
df_acc(results['fgsm'])

0.25      0.50      0.75      1.00      1.25
emodb   All     0.155317  0.317802  0.633214  0.770609  0.784946
        Female  0.193487  0.358238  0.620690  0.758621  0.783525
        Male    0.092063  0.250794  0.653968  0.790476  0.787302
emovo   All     0.059777  0.262411  0.839919  0.883485  0.888551
        Female  0.058116  0.228457  0.827655  0.871744  0.877756
        Male    0.061475  0.297131  0.852459  0.895492  0.899590
ravdess All     0.145729  0.391960  0.598548  0.656616  0.684534
        Female  0.154018  0.390625  0.613839  0.679688  0.707589
        Male    0.137430  0.393296  0.583240  0.633520  0.661453

In [21]:
epss = [0.25]
conf2 = {d: {e: BasicIterativeMethod(estimator=classifiers[d],
                                     eps=e,
                                     eps_step=0.1,
                                     batch_size=bs,
                                     max_iter=100,
                                     verbose=True)
             for e in epss}
         for d in datasets}

adv = attack(conf2, 'bim')

Attacking emodb with bim, parameter 0.25


PGD - Batches: 0it [00:00, ?it/s]

Attack get 24.919155597686768 seconds to complete
	Whole test set
		loss: 20.68187, accuracy: 0.05974, avg perturbation: 0.16373

	Only male data
		loss: 21.44783, accuracy: 0.05714, avg perturbation: 0.16300

	Only female data
		loss: 20.21966, accuracy: 0.06130, avg perturbation: 0.16416

_____________________________
Attacking emovo with bim, parameter 0.25


PGD - Batches: 0it [00:00, ?it/s]

Attack get 28.455023765563965 seconds to complete
	Whole test set
		loss: 29.34666, accuracy: 0.06079, avg perturbation: 0.15901

	Only male data
		loss: 30.27676, accuracy: 0.05738, avg perturbation: 0.15816

	Only female data
		loss: 28.43706, accuracy: 0.06413, avg perturbation: 0.15984

_____________________________
Attacking ravdess with bim, parameter 0.25


PGD - Batches: 0it [00:00, ?it/s]

Attack get 51.90227937698364 seconds to complete
	Whole test set
		loss: 31.04280, accuracy: 0.05807, avg perturbation: 0.15380

	Only male data
		loss: 31.55669, accuracy: 0.06145, avg perturbation: 0.15229

	Only female data
		loss: 30.52949, accuracy: 0.05469, avg perturbation: 0.15532

_____________________________


In [22]:
df_acc(results['bim'])

0.25
emodb   All     0.059737
        Male    0.057143
        Female  0.061303
emovo   All     0.060790
        Male    0.057377
        Female  0.064128
ravdess All     0.058068
        Male    0.061453
        Female  0.054688

In [23]:
iterations = [5]
conf3 = {d: {i: DeepFool(classifier=classifiers[d],
                         nb_grads=5,
                         epsilon=1e-05,  # to help convergence and avoid being stuck to 0
                         max_iter=i,
                         batch_size=bs,
                         verbose=True)
             for i in iterations}
         for d in datasets}

adv = attack(conf3, 'df')

Attacking emodb with df, parameter 5


DeepFool:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 9.343069314956665 seconds to complete
	Whole test set
		loss: 3.22380, accuracy: 0.06213, avg perturbation: 1.63845

	Only male data
		loss: 3.57804, accuracy: 0.06032, avg perturbation: 1.49885

	Only female data
		loss: 3.01004, accuracy: 0.06322, avg perturbation: 1.72269

_____________________________
Attacking emovo with df, parameter 5


DeepFool:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 9.438507080078125 seconds to complete
	Whole test set
		loss: 5.33874, accuracy: 0.05471, avg perturbation: 1.29036

	Only male data
		loss: 5.20357, accuracy: 0.05738, avg perturbation: 1.43760

	Only female data
		loss: 5.47093, accuracy: 0.05210, avg perturbation: 1.14637

_____________________________
Attacking ravdess with df, parameter 5


DeepFool:   0%|          | 0/7 [00:00<?, ?it/s]

Attack get 17.785712242126465 seconds to complete
	Whole test set
		loss: 3.62052, accuracy: 0.05639, avg perturbation: 1.32913

	Only male data
		loss: 3.75198, accuracy: 0.06145, avg perturbation: 1.15660

	Only female data
		loss: 3.48921, accuracy: 0.05134, avg perturbation: 1.50147

_____________________________


In [24]:
df_acc(results['df'])

5
emodb   All     0.062127
        Male    0.060317
        Female  0.063218
emovo   All     0.054711
        Male    0.057377
        Female  0.052104
ravdess All     0.056393
        Male    0.061453
        Female  0.051339

In [28]:
# amount of pertubration
thetas = [1]
conf4 = {d: {t: SaliencyMapMethod(classifier=classifiers[d],
                                  theta=t, # amount of perturbation introduced per step
                                  gamma=1, #  maximum fraction of features being perturbed
                                  batch_size=bs,
                                  verbose=True)                           
             for t in thetas}
         for d in datasets}

adv = attack(conf4, 'jsma')

Attacking emodb with jsma, parameter 1


JSMA:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 303.91102504730225 seconds to complete
	Whole test set
		loss: 1.14330, accuracy: 0.02509, avg perturbation: 0.00315

	Only male data
		loss: 1.21690, accuracy: 0.02857, avg perturbation: 0.00239

	Only female data
		loss: 1.09889, accuracy: 0.02299, avg perturbation: 0.00362

_____________________________
Attacking emovo with jsma, parameter 1


JSMA:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 96.52399682998657 seconds to complete
	Whole test set
		loss: 1.22358, accuracy: 0.03546, avg perturbation: 0.00154

	Only male data
		loss: 1.15288, accuracy: 0.02459, avg perturbation: 0.00149

	Only female data
		loss: 1.29272, accuracy: 0.04609, avg perturbation: 0.00158

_____________________________
Attacking ravdess with jsma, parameter 1


JSMA:   0%|          | 0/7 [00:00<?, ?it/s]

Attack get 346.43396639823914 seconds to complete
	Whole test set
		loss: 1.09161, accuracy: 0.01619, avg perturbation: 0.00254

	Only male data
		loss: 1.10926, accuracy: 0.01899, avg perturbation: 0.00216

	Only female data
		loss: 1.07398, accuracy: 0.01339, avg perturbation: 0.00291

_____________________________


In [29]:
df_acc(results['jsma'])

1
emodb   All     0.025090
        Male    0.028571
        Female  0.022989
emovo   All     0.035461
        Male    0.024590
        Female  0.046092
ravdess All     0.016192
        Male    0.018994
        Female  0.013393

In [30]:
cs = [1e-04]
conf7 = {d: {c: CarliniLInfMethod(classifier=classifiers[d],
                                  max_iter=10,
                                  learning_rate=0.01,
                                  initial_const=c,
                                  largest_const=2e01,
                                  decrease_factor=0.8,
                                  const_factor=4,
                                  batch_size=bs,
                                  verbose=True)
             for c in cs}
         for d in datasets}

adv = attack(conf7, 'cwinf')

Attacking emodb with cwinf, parameter 0.0001


C&W L_inf:   0%|          | 0/837 [00:00<?, ?it/s]

Attack get 4330.973206996918 seconds to complete
	Whole test set
		loss: 2.76422, accuracy: 0.06332, avg perturbation: 0.05518

	Only male data
		loss: 2.97496, accuracy: 0.05714, avg perturbation: 0.05217

	Only female data
		loss: 2.63706, accuracy: 0.06705, avg perturbation: 0.05700

_____________________________
Attacking emovo with cwinf, parameter 0.0001


C&W L_inf:   0%|          | 0/987 [00:00<?, ?it/s]

Attack get 5233.7448971271515 seconds to complete
	Whole test set
		loss: 4.26115, accuracy: 0.06079, avg perturbation: 0.03555

	Only male data
		loss: 4.53054, accuracy: 0.05738, avg perturbation: 0.03521

	Only female data
		loss: 3.99770, accuracy: 0.06413, avg perturbation: 0.03588

_____________________________
Attacking ravdess with cwinf, parameter 0.0001


C&W L_inf:   0%|          | 0/1791 [00:00<?, ?it/s]

Attack get 10467.29465508461 seconds to complete
	Whole test set
		loss: 2.93579, accuracy: 0.06309, avg perturbation: 0.02863

	Only male data
		loss: 3.04112, accuracy: 0.06816, avg perturbation: 0.02638

	Only female data
		loss: 2.83058, accuracy: 0.05804, avg perturbation: 0.03087

_____________________________


In [31]:
df_acc(results['cwinf'])

0.0001
emodb   All     0.063321
        Male    0.057143
        Female  0.067050
emovo   All     0.060790
        Male    0.057377
        Female  0.064128
ravdess All     0.063093
        Male    0.068156
        Female  0.058036

In [32]:
# amount of pertubration
thetas = [10]
conf8 = {d: {t: PixelAttack(classifier=classifiers[d],
                            max_iter=20, # max iter evolution strategy
                            es=1,    # DE as CMA-ES not aviable
                            th=t,
                            verbose=True)                           
             for t in thetas}
         for d in datasets}

adv = attack(conf8, 'pa')

Attacking emodb with pa, parameter 10


Pixel threshold: 0it [00:00, ?it/s]

Attack get 2155.0962631702423 seconds to complete
	Whole test set
		loss: 0.83253, accuracy: 0.56272, avg perturbation: 0.00054

	Only male data
		loss: 1.09498, accuracy: 0.48889, avg perturbation: 0.00061

	Only female data
		loss: 0.67415, accuracy: 0.60728, avg perturbation: 0.00050

_____________________________
Attacking emovo with pa, parameter 10


Pixel threshold: 0it [00:00, ?it/s]

Attack get 1113.456562280655 seconds to complete
	Whole test set
		loss: 1.92566, accuracy: 0.31712, avg perturbation: 0.00082

	Only male data
		loss: 1.89085, accuracy: 0.34426, avg perturbation: 0.00080

	Only female data
		loss: 1.95970, accuracy: 0.29058, avg perturbation: 0.00083

_____________________________
Attacking ravdess with pa, parameter 10


Pixel threshold: 0it [00:00, ?it/s]

Attack get 2300.9179978370667 seconds to complete
	Whole test set
		loss: 2.12742, accuracy: 0.38582, avg perturbation: 0.00087

	Only male data
		loss: 2.81658, accuracy: 0.28268, avg perturbation: 0.00102

	Only female data
		loss: 1.43903, accuracy: 0.48884, avg perturbation: 0.00071

_____________________________


In [33]:
df_acc(results['pa'])

10
emodb   All     0.562724
        Male    0.488889
        Female  0.607280
emovo   All     0.317123
        Male    0.344262
        Female  0.290581
ravdess All     0.385818
        Male    0.282682
        Female  0.488839

In [26]:
# amount of pertubration
eps = [5]
conf9 = {d: {e: BoundaryAttack(estimator=classifiers[d],
                               delta=0.01,     # dynamic adjusted
                               epsilon=0.01,   # dynamic adjusted
                               step_adapt=0.5, # suggested in the paper
                               max_iter=100,
                               num_trial=3,
                               sample_size=10,
                               targeted=False,
                               batch_size=bs,
                               verbose=True)                       
             for e in eps}
         for d in datasets}
    
adv = attack(conf9, 'ba')

Attacking emodb with ba, parameter 5


Boundary attack:   0%|          | 0/837 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

...

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Attack get 1684.1790096759796 seconds to complete
	Whole test set
		loss: 1.44094, accuracy: 0.03238, avg perturbation: 0.93386

	Only male data
		loss: 1.53043, accuracy: 0.03240, avg perturbation: 1.02818

	Only female data
		loss: 1.35156, accuracy: 0.03237, avg perturbation: 0.83964

_____________________________


In [27]:
df_acc(results['ba'])

5
emodb   All     0.204301
        Male    0.225397
        Female  0.191571
emovo   All     0.040527
        Male    0.038934
        Female  0.042084
ravdess All     0.032384
        Male    0.032402
        Female  0.032366